<a href="https://colab.research.google.com/github/Corassini/Curso_deep_learning/blob/main/Exercicio_selecao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Abrindo o Arquivo CISI

In [3]:
!tar xf cisi.tar.gz

In [4]:
!pwd

/content


In [5]:
!ls

CISI.ALL  CISI.BLN  CISI.QRY  CISI.REL	cisi.tar.gz  sample_data


### Lendo os arquivos CISI e separando em diferentes variáveis

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

import nltk
nltk.download('stopwords')
nltk.download('punkt')


stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer() 
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
with open('/content/CISI.ALL','r') as f:
    #reorganizando a entrada do arquivo .ALL
    #agrupando as linhas que começam com ponto e removendo espaços
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")

doc_set = {}
doc_id = ""
doc_text = ""

# Se a linha começa com .I extraio o ID do documento para facilidar a analise posterior
# Se a linha começa com .X é o resumo inteiro, atribuo no dicionario doc_set com o ID anterior
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip() 
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " 

#Criação de uma lista documents
#tratamento de stemmer (radical), Stopwords e separa-las dentro da lista doc_token
documents = []
for key, value in doc_set.items():
  documents.append(value)
doc_token = []
for item in documents:
  doc_token.append([stemmer.stem(token) for token in tokenizer.tokenize(item.lower()) if token not in stop_words])

In [9]:
#Utilização da mesma arquitetura do .ALL
# Dado que o Arquivo .QRY será as buscas feitas para encontrar os resumos do .ALL
with open('/content/CISI.QRY', 'r') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
      
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip() 
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""


queries = []
for key, value in qry_set.items():
  queries.append(value)
queries_token = []
for item in queries:
  queries_token.append([stemmer.stem(token) for token in tokenizer.tokenize(item.lower()) if token not in stop_words])

### Implementando o BM25

In [10]:
from gensim.summarization.bm25 import BM25
from gensim.summarization.bm25 import get_bm25_weights

In [11]:
#Utilização do BM25 - ChatGPT 
# Colocando o input como os documentos tratados acima
model_bm25 = BM25(doc_token)

In [12]:
#Calculando o IDF médio
idf_values = model_bm25.idf.values()
idf_sum = 0
for value in idf_values:
  idf_sum += value
idf_mean = idf_sum / len(idf_values)

In [13]:
# Calculando o score
# Utilizando os queries_token tratados acima
# Get_scores já faz o batimento das querys com os resumos
scores = []
for query in queries_token:
  query_scores = model_bm25.get_scores(query,idf_mean)
  scores.append(query_scores)

In [18]:
#Query a query trago os documentos por ordem reversa (do maior para o menor) do score calculado acima
# Ou seja, trazer os resumos que são mais ligados àquela query
for i, query in enumerate(queries_token):
    ranked_documents = sorted(range(len(scores[i])), key=lambda j: scores[i][j], reverse=True)
    print("Consulta: ", query)
    print("Documentos mais relevantes: ", [k+1 for k in ranked_documents[:10]])

Consulta:  ['problem', 'concern', 'make', 'descript', 'titl', 'difficulti', 'involv', 'automat', 'retriev', 'articl', 'approxim', 'titl', 'usual', 'relev', 'content', 'articl', 'titl']
Documentos mais relevantes:  [429, 722, 1299, 759, 65, 76, 603, 38, 1421, 711]
Consulta:  ['actual', 'pertin', 'data', 'oppos', 'refer', 'entir', 'articl', 'retriev', 'automat', 'respons', 'inform', 'request']
Documentos mais relevantes:  [1138, 309, 420, 597, 797, 790, 488, 1071, 1327, 451]
Consulta:  ['inform', 'scienc', 'give', 'definit', 'possibl']
Documentos mais relevantes:  [1181, 540, 469, 1179, 1133, 1077, 1235, 914, 1277, 1142]
Consulta:  ['imag', 'recognit', 'method', 'automat', 'transform', 'print', 'text', 'comput', 'readi', 'form']
Documentos mais relevantes:  [179, 601, 420, 77, 790, 175, 565, 746, 72, 739]
Consulta:  ['special', 'train', 'ordinari', 'research', 'businessmen', 'need', 'proper', 'inform', 'manag', 'unobstruct', 'use', 'inform', 'retriev', 'system', 'problem', 'like', 'encou

### Calculando a acurácia

In [15]:
#Criação de um dicionário (rel_set) trazendo quais documentos (.ALL) devem estar ligados à qual busca (.QRY)

rel_set = {}
with open('/content/CISI.REL', 'r') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0] 
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

In [16]:
#Tratanndo o rel_set -- chamando de ground truth para fazer a compação da predição do modelo (model_return) com o que de fato deveria ser entregue
model_return = []
model_return.append('')
ground_truth = []
ground_truth.append('')

for i, query in enumerate(queries_token):
  ranked_documents = sorted(range(len(scores[i])), key=lambda j: scores[i][j], reverse=True)
  if str(i+1) in rel_set:
    model_return.append([k+1 for k in ranked_documents[:len(rel_set[str(i+1)])]])
    ground_truth.append([int(k) for k in rel_set[str(i+1)]])
  else:
    model_return.append([])
    ground_truth.append([])

In [17]:
# Trazendo a variável precision para saber quantos artigos foram acertados na busca
# Há um tratamento de que o retorno o modelo scora e trás todos os artigos por ordem de score
# Limitei a quantidade de artigos de volta pela quantidade que está no rel_set - dessa maneira se torna mais comparável - mas, ainda acredito que não seja a melhor forma

precision_list = []
for i in range(1,len(queries)+1):
  acerto =0
  for k in model_return[i]:
    if k in ground_truth[i]:
      acerto += 1
  if ground_truth[i] != []:
    precision_list.append(acerto/len(ground_truth[i]))
  else:
    # Colocando o -9 para as buscas que não deveriam de retornar nada
    precision_list.append(-9)

precision_final =0
count = 0
for i in range(len(precision_list)):
  if precision_list[i] != -9:
    precision_final +=precision_list[i]
    count +=1

precision_final_2 = precision_final/count
print(precision_final_2)
print( f"Precisão final de: {precision_final_2}")

0.2498303025107418
Precisão final de: 0.2498303025107418
